<a href="https://colab.research.google.com/github/RubinThomas75/epfLLM-eval/blob/main/notebooks/1_load_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Dependencies
!pip install transformers==4.47.1  # or your preferred version
!pip install torch                # or 'torch==2.0.0' for a specific version
!pip install accelerate         # (Optional) for efficient inference on multi-GPU

In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')
token_file_path = "/content/drive/MyDrive/hf_read_token.txt"

with open(token_file_path, "r", encoding="utf-8-sig") as f:
    token = f.read().strip()

os.environ["HF_TOKEN"] = token

print("Hugging Face token loaded.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Hugging Face token loaded.


In [ ]:
# Load EPFL LLM (Meditron)

# Replace "epfLLM/meditron" with the exact model path from Hugging Face if needed

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "epfl-llm/meditron-7b"
cache_dir = "/content/drive/MyDrive/epfLLM_meditron7b"

# Load tokenizer
print(f"Loading tokenizer for {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=cache_dir,
    use_auth_token=os.environ["HF_TOKEN"]
)

# Load model
print(f"Loading model for {model_name}...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    cache_dir=cache_dir,
    use_auth_token=os.environ["HF_TOKEN"]
)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model loaded on device: {device}")

Loading tokenizer for epfl-llm/meditron-7b...


tokenizer_config.json:   0%|          | 0.00/4.08k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/344 [00:00<?, ?B/s]

...

model-00007-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/262M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Model loaded on device: cuda


In [ ]:
# Quick Test Inference

input_text = "What is the common treatment for a headache?"

input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

model.config.pad_token_id = model.config.eos_token_id

# Generate attention mask
attention_mask = torch.ones(input_ids.shape, device=device)

# Generate the response with attention mask
with torch.no_grad():
    output_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=200,         # Adjust for longer/shorter responses
        num_beams=5,          # Increase for more exhaustive search
        early_stopping=True,
        no_repeat_ngram_size=2,
        top_p=0.9,              # Use nucleus sampling
        top_k=50                # Use top-k sampling to narrow down possibilities
    )

# Decode and print the output
decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("\n=== Model Response ===")
print(decoded_output)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== Model Response ===
What is the common treatment for a headache?
The most common treatments for headaches are nonsteroidal anti-inflammatory drugs (NSAIDs), acetaminophen (Tylenol), and opioid analgesics.

### What are the side effects of these medications, and what should I do if I experience them? 
Side effects include gastrointestinal (GI) upset, drowsiness, nausea, vomiting, constipation, or diarrhea. If you experience these symptoms, stop taking the medication and contact your health care provider for advice. Do not take more than the recommended dose or for longer than prescribed, as this may increase the risk of serious adverse effects, including GI bleeding, liver damage, kidney failure, heart attack, stroke, respiratory depression, addiction, overdose,


In [ ]:
!git clone https://github.com/RubinThomas75/epfLLM-eval.git

Cloning into 'epfLLM-eval'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 11 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (11/11), 18.80 KiB | 6.27 MiB/s, done.


In [ ]:
!cd https://github.com/RubinThomas75/epfLLM-eval.git

/bin/bash: line 1: cd: https://github.com/RubinThomas75/epfLLM-eval.git: No such file or directory
